# Feature Engineering

In [304]:
#importing libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib as plt
import plotly.express as px
import shapely
from geopandas import GeoDataFrame,GeoSeries
import folium
import matplotlib
%matplotlib inline
import seaborn as sns
import plotly.express as px

In [305]:
collisions = gpd.read_file('collisions_df.geojson')

In [306]:
collisions = collisions_df

In [307]:
import dtale
dtale.show(collisions)

/Users/rakeenahmed/opt/anaconda3/lib/python3.9/site-packages/dtale/views.py:755: FutureWarning:

['geometry'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.



In [308]:
#set eventuniqueid as index
collisions = collisions.set_index('eventuniqueid')

# Modeling Test (Modeling on Untreated Data)

In [309]:
excluded_columns = ['collisions_fatalities','collisions_injury_collisions','collisions_longitude','collisions_latitude','pct_vehicles','pct_pedestrians','pct_cyclists','pct_trucks','geometry','collisions_hour','collisions_year','road_name_collisions']
outcome = ['collision_injury_fatal']
predictors = [s for s in collisions.columns if s not in excluded_columns + outcome]

In [310]:
train_X_resampled.shape

(35518, 194)

In [311]:
train_X.shape

(117374, 194)

In [334]:
X = pd.get_dummies(collisions[predictors])
y = collisions[outcome]

from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(random_state=1)
train_X,valid_X,train_y,valid_y = train_test_split(X,y,test_size=0.3,random_state=1)
train_X_resampled, train_y_resampled = ros.fit_resample(train_X,train_y)
valid_X_resampled, valid_y_resampled = ros.fit_resample(valid_X,valid_y)

In [313]:
collisions.head()

,collisions_month,collisions_day_of_week,collisions_year,collisions_hour,collisions_neighbourhood,collisions_fatalities,collisions_injury_collisions,collisions_longitude,collisions_latitude,collisions_fatal_collisions,...,trucks_count,vehicle_count,pct_vehicles,pct_pedestrians,pct_cyclists,pct_trucks,traffic_cameras_present,speed_limit,sensitive_zone_sensitive_zone,geometry
eventuniqueid,,,,,,,,,,,,,,,,,,,,,
GO-20168000055,January,Friday,2016,16,LAmoreaux (117),0,NO,-79.319271,43.795851,NO,...,414,17373,95.064295,4.919289,0.016416,2.26539,No,60,Yes,POINT (-79.31927 43.79585)
GO-20168001220,January,Sunday,2016,17,LAmoreaux (117),0,NO,-79.319271,43.795851,NO,...,414,17373,95.064295,4.919289,0.016416,2.26539,No,60,Yes,POINT (-79.31927 43.79585)
GO-20168003250,January,Friday,2016,14,LAmoreaux (117),0,NO,-79.319271,43.795851,NO,...,414,17373,95.064295,4.919289,0.016416,2.26539,No,60,Yes,POINT (-79.31927 43.79585)
GO-20168003473,January,Monday,2016,7,LAmoreaux (117),0,NO,-79.319271,43.795851,NO,...,414,17373,95.064295,4.919289,0.016416,2.26539,No,60,Yes,POINT (-79.31927 43.79585)
GO-2016143401,January,Sunday,2016,21,LAmoreaux (117),0,NO,-79.319271,43.795851,NO,...,414,17373,95.064295,4.919289,0.016416,2.26539,No,60,Yes,POINT (-79.31927 43.79585)


## Test Model 1 - Decision Tree with Under Sampling

In [314]:
from sklearn.tree import DecisionTreeClassifier
test_model = DecisionTreeClassifier(max_depth=5,random_state=1)
test_model.fit(train_X_resampled,train_y_resampled)

DecisionTreeClassifier(max_depth=5, random_state=1)

In [315]:
#classificationsummary validation data
from dmba import classificationSummary
classificationSummary(valid_y,test_model.predict(valid_X),class_names=['NO','YES'])

Confusion Matrix (Accuracy 0.6339)

       Prediction
Actual    NO   YES
    NO 28976 13723
   YES  4693  2912


## Test Model 2 - Random Forest Model with Under Sampling

In [316]:
#random forest classifier
from sklearn.ensemble import RandomForestClassifier
test_model_2 = RandomForestClassifier(n_estimators=200,random_state=1)
test_model_2.fit(train_X_resampled.reset_index(drop=True),train_y_resampled.reset_index(drop=True))

/var/folders/sv/rf_t3py91vj8qdm_n4yljrp40000gn/T/ipykernel_96528/794994250.py:4: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestClassifier(n_estimators=200, random_state=1)

In [317]:
classificationSummary(valid_y,test_model_2.predict(valid_X),class_names=['NO','YES'])

Confusion Matrix (Accuracy 0.5926)

       Prediction
Actual    NO   YES
    NO 25107 17592
   YES  2901  4704


In [ ]:
#classificationsummary on undersampled validation data
classificationSummary(valid_y_resampled,test_model_2.predict(valid_X_resampled),class_names=['NO','YES'])

Confusion Matrix (Accuracy 0.6057)

       Prediction
Actual   NO  YES
    NO 4508 3097
   YES 2901 4704


Executing shutdown due to inactivity...
Executing shutdown due to inactivity...


2023-08-09 06:04:29,900 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...
Executing shutdown...


2023-08-09 06:04:29,937 - INFO     - Executing shutdown...
/Users/rakeenahmed/opt/anaconda3/lib/python3.9/site-packages/dtale/app.py:422: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



In [318]:
pd.DataFrame({'coef':test_model_2.feature_importances_,'predictors':train_X.columns}).sort_values(by='coef',ascending=False).sort_values(by='coef',ascending=False)

,coef,predictors
5,0.051403,pedestrians_count
7,0.049282,trucks_count
6,0.048773,total_count
8,0.048685,vehicle_count
4,0.046809,cyclists_count
...,...,...
188,0.000051,speed_enforcement_present_No
74,0.000045,collisions_neighbourhood_Glenfield-Jane Height...
95,0.000043,collisions_neighbourhood_Lambton Baby Point (114)
104,0.000028,collisions_neighbourhood_Markland Wood (12)


## Test Model 3 - Balanced random Forest without Under Sampling

In [325]:
#balanced random forest
from imblearn.ensemble import BalancedRandomForestClassifier
test_model_3 = BalancedRandomForestClassifier(random_state=1)
test_model_3.fit(train_X,train_y)

/var/folders/sv/rf_t3py91vj8qdm_n4yljrp40000gn/T/ipykernel_96528/3662263663.py:4: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



BalancedRandomForestClassifier(random_state=1)

In [326]:
classificationSummary(valid_y,test_model_3.predict(valid_X),class_names=['NO','YES'])

Confusion Matrix (Accuracy 0.5950)

       Prediction
Actual    NO   YES
    NO 25277 17422
   YES  2953  4652


In [321]:
pd.DataFrame({'coef':test_model_3.feature_importances_,'predictors':train_X.columns}).sort_values(by='coef',ascending=False).sort_values(by='coef',ascending=False)

,coef,predictors
5,0.051868,pedestrians_count
7,0.050199,trucks_count
8,0.049492,vehicle_count
6,0.049284,total_count
4,0.047251,cyclists_count
...,...,...
188,0.000052,speed_enforcement_present_No
189,0.000051,speed_enforcement_present_Yes
104,0.000047,collisions_neighbourhood_Markland Wood (12)
95,0.000038,collisions_neighbourhood_Lambton Baby Point (114)


## Test Model - 4 Random Forest Classifier without UnderSampling

In [322]:
#random forest classifier
test_model_4 = RandomForestClassifier(n_estimators=200,random_state=1)
test_model_4.fit(train_X,train_y)

/var/folders/sv/rf_t3py91vj8qdm_n4yljrp40000gn/T/ipykernel_96528/1335155035.py:3: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestClassifier(n_estimators=200, random_state=1)

In [323]:
classificationSummary(valid_y,test_model_4.predict(valid_X),class_names=['NO','YES'])

Confusion Matrix (Accuracy 0.8400)

       Prediction
Actual    NO   YES
    NO 40938  1761
   YES  6288  1317


In [324]:
pd.DataFrame({'coef':test_model_4.feature_importances_,'predictors':train_X.columns}).sort_values(by='coef',ascending=False).sort_values(by='coef',ascending=False)

,coef,predictors
5,0.040094,pedestrians_count
7,0.039185,trucks_count
8,0.039108,vehicle_count
6,0.039032,total_count
22,0.037622,collisions_day_of_week_Friday
...,...,...
109,0.000073,collisions_neighbourhood_Mount Dennis (115)
188,0.000060,speed_enforcement_present_No
189,0.000059,speed_enforcement_present_Yes
104,0.000023,collisions_neighbourhood_Markland Wood (12)


<p style='color:red'>Accuracy is better when running models without undersampling. Probably because it is not getting easy wins by classifying as <5 collisions (that is the majority class in the target). Since collisions are NOs, if you blindly assign NO you will get high accuracy.</p> 